In [1]:
from metavision_core.event_io import EventsIterator, load_events
import tonic
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import random
from IPython.utils import io
# import sdl2.ext

import models
import utils

In [2]:
# %matplotlib inline
# %matplotlib ipympl
# %matplotlib tk
plt.rc('animation', html='jshtml')
plt.rcParams["image.cmap"] = "bone"

In [3]:
in_data = tonic.datasets.NMNIST(save_to="../data", train=True) # (34, 34, 2) 3
# in_data = tonic.datasets.POKERDVS(save_to="../data",train=True) # (35, 35, 2) 3
# in_data = tonic.datasets.DVSGesture(save_to="../data",train=True) # (128, 128, 2) 2
# in_data = tonic.datasets.ASLDVS(save_to="../data", train=True) # (240, 180, 2) 3

in_data.sensor_size

(34, 34, 2)

In [4]:
noise = tonic.transforms.UniformNoise(
    sensor_size=in_data.sensor_size[:-1]+(1,),
    n=int(4172),
)
# to_frame = tonic.transforms.ToFrame(in_data.sensor_size[:-1]+(1,), time_window=100)
to_frame = tonic.transforms.ToFrame(in_data.sensor_size[:-1]+(1,), n_time_bins=100)

In [5]:
events, label = in_data[random.randint(0, len(in_data)-1)]
events = events.squeeze()
events = np.array(list(filter(lambda ev: ev[3]==0, events)))
# print(events)
print(label)

true_frames = torch.tensor(to_frame(events)).float()#.cuda(0)
in_frames = torch.tensor(to_frame(noise(events))).float()#.cuda(0)

6


In [6]:
# ani.save("./img/nmnist-9.gif", writer="imagemagick")

In [7]:
net = models.Conv_FastLIF(models.kernels.plus3even)#.cuda(0)
with torch.inference_mode():
    out_frames = net(in_frames)

AttributeError: module 'models' has no attribute 'Kernels'

In [ ]:
in_frames = utils.frame_merge(in_frames,5).clamp_max(1).cpu()
out_frames = utils.frame_merge(out_frames,5).clamp_max(1).cpu()
true_frames = utils.frame_merge(true_frames,5).clamp_max(1).cpu()

In [ ]:
loss=nn.BCELoss()
print("noisy loss:"+ str(loss(in_frames,true_frames)))
print("denoised loss:"+ str(loss(out_frames,true_frames)))

noisy loss:tensor(15.4758)
denoised loss:tensor(3.3002)


In [ ]:
with io.capture_output() as captured:
    anim = tonic.utils.plot_animation(np.concatenate((in_frames,out_frames,true_frames),axis=3))
display(anim)

In [ ]:
# anim.save("./img/dataline3.gif", fps=300)